In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')

# Set path to root directory so we can access core, tasks, data, etc.
os.chdir('..')

In [2]:
import numpy
def smooth(x,window_len=11,window='hanning'):
    if x.ndim != 1:
            raise ValueError, "smooth only accepts 1 dimension arrays."
    if x.size < window_len:
            raise ValueError, "Input vector needs to be bigger than window size."
    if window_len<3:
            return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
            raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
    s=numpy.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
    if window == 'flat': #moving average
            w=numpy.ones(window_len,'d')
    else:  
            w=eval('numpy.'+window+'(window_len)')
    y=numpy.convolve(w/w.sum(),s,mode='same')
    return y[window_len:-window_len+1]

In [40]:
vid = pd.read_csv('data/videos/@Animated/@NewDisney/Feast/preds_old_torch/posneg.csv',
                  header=None, names=['Positive', 'Negative'])
smoothed = smooth(vid.Positive.values, window_len=48)

pd.Series(smoothed).plot()
plt.show()

In [39]:
# sent_biclass_4 might be best? Starts to overfit after?
vid = pd.read_csv('data/videos/@Animated/@NewDisney/Feast/preds/sent_biclass_10.csv')
smoothed = smooth(vid.pos.values, window_len=48)
pd.Series(smoothed).plot()
plt.show()